In [205]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn 
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [206]:
df = pd.read_csv('housing.csv')
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [207]:
df.isna().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

In [208]:
X, y = df.drop(columns=['median_house_value', 'ocean_proximity', 'total_bedrooms']).values, df['median_house_value'].values
X.shape

(20640, 7)

In [209]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [210]:
def normalize(tensor):
    mean = tensor.mean()
    std = tensor.std()
    return (tensor - mean) / std, std, mean

In [211]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32) 
y_train_tensor = torch.tensor(y_train, dtype=torch.float32) 

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

X_train_norm, mean_train, std_train = normalize(X_train_tensor).to(device)
X_train_norm = X_train_norm.to(device)
X_test_norm = (X_test_tensor - mean_train) / std_train

y_train_norm, mean, std = normalize(y_train_tensor)
y_train_norm = y_train_norm.to(device)
y_test_norm = (y_test_tensor - mean) / std

In [212]:
class MLPWithDropout(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim, dropout_rate=0.2):
        super(MLPWithDropout, self).__init__()
        
        hidden_dims = [input_dim] + hidden_dims

        self.layers = []
        for i in range(1, len(hidden_dims)):
            self.layers.append(nn.Sequential(
            nn.Linear(hidden_dims[i-1], hidden_dims[i]),
            nn.ReLU(),
            nn.Dropout(dropout_rate)))

        self.output_layer = nn.Linear(hidden_dims[-1], output_dim)

    
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return self.output_layer(x)
    
model = MLPWithDropout(7, [1000, 2000, 2000, 1000], 1).to(device)


In [213]:
y_test_norm

tensor([ 1.8564e+00,  8.4858e-01,  2.4301e-02,  ...,  2.7443e-01,
        -1.7748e-03,  1.4705e+00])

In [ ]:
# Paramètres
batch_size = 256
epochs = 200
learning_rate = 0.01
patience = 10  # Nombre d'époques sans amélioration avant d'arrêter
best_val_loss = float('inf')  # Initialiser la meilleure perte de validation à l'infini
early_stop_counter = 0  # Compteur pour suivre le nombre d'époques sans amélioration


# DataLoader pour gérer les batches
train_dataset = TensorDataset(X_train_norm, y_train_norm)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

model.train()  # Mode entraînement

# Définition de la fonction de perte et de l'optimiseur
criterion = nn.MSELoss()  # Régression -> Mean Squared Error
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

# Boucle d'entraînement avec early stopping
for epoch in range(epochs):
    epoch_loss = 0.0
    for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
        # Remise à zéro des gradients
        optimizer.zero_grad()

        # Forward pass
        y_pred = model(X_batch)
        # Calcul de la perte
        loss = criterion(y_pred, y_batch)

        # Backward pass et optimisation
        loss.backward()
        optimizer.step()

        # Accumuler la perte
        epoch_loss += loss.item()

    # Calcul de la perte moyenne pour cette époque
    avg_epoch_loss = epoch_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {avg_epoch_loss:.4f}")
    
    # Early stopping
    if avg_epoch_loss < best_val_loss:
        best_val_loss = avg_epoch_loss
        early_stop_counter = 0  # Réinitialiser le compteur d'amélioration
    else:
        early_stop_counter += 1

    # Vérifier si le critère de patience est atteint
    if early_stop_counter >= patience:
        print(f"Early stopping at epoch {epoch + 1}.")
        break


c:\Users\USER\miniconda3\Lib\site-packages\torch\nn\modules\loss.py:608: UserWarning:

Using a target size (torch.Size([256])) that is different to the input size (torch.Size([256, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.

c:\Users\USER\miniconda3\Lib\site-packages\torch\nn\modules\loss.py:608: UserWarning:

Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.



Epoch 1/200, Loss: 1.0044
Epoch 2/200, Loss: 1.0026
Epoch 3/200, Loss: 1.0073


In [181]:
X_test_tensor = torch.tensor(X_test_norm, dtype=torch.float32)

In [182]:
y_train

array([ 78800., 181100., 195700., ...,  82800., 112500., 162500.])

In [183]:
y_train

array([ 78800., 181100., 195700., ...,  82800., 112500., 162500.])

In [184]:
model.train()
model(X_train_tensor)

tensor([[2551.8594],
        [1720.8389],
        [1611.1016],
        ...,
        [1561.5289],
        [2428.8772],
        [2174.5776]], grad_fn=<AddmmBackward0>)

In [ ]:
import torch
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Créer un tenseur pour X_test_norm (normalisé)
with torch.no_grad():
        for name, param in model.named_parameters():
            if 'weight' in name:  # On ajuste uniquement les poids, pas les biais
                param.data *= 1 / 0.2
                print(f"Ajustement des poids pour la couche {name}")


# Effectuer des prédictions avec MC-Dropout
y_pred_stack = np.stack([model(X_test_tensor).detach().numpy() for _ in range(10)])

# Créer un DataFrame avec les vraies valeurs
predictions_df = pd.DataFrame({
    'X_test': X_test_norm.flatten(),
    'True Values': y_test
})

# Ajouter les prédictions dans le DataFrame de manière dynamique
for i in range(10):
    predictions_df[f'Prediction {i+1}'] = y_pred_stack[i].flatten()




# Création du graphique
plt.figure(figsize=(15, 6))

# Tracer les vraies valeurs
sns.scatterplot(x='X_test', y='True Values', data=predictions_df, color='blue', label='True Values')

# Tracer toutes les prédictions avec transparence
for i in range(1, 11):
    sns.scatterplot(x='X_test', y=f'Prediction {i}', data=predictions_df, color='red', alpha=0.1)

# Ajouter un titre et une légende
plt.title('MC-Dropout Predictions vs True Values')
plt.legend()

# Afficher le graphique
plt.show()


Ajustement des poids pour la couche output_layer.weight


ValueError: All arrays must be of the same length